In [2]:
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

In [3]:
df_2020 = pd.read_csv('base_2020.csv', sep=';')
df_2021 = pd.read_csv('base_2021.csv', sep=';')
df_2022 = pd.read_csv('base_2022.csv', sep=';')
df = pd.read_csv('base_full.csv', sep=';')

In [5]:
def plot_instituicao_ensino(df, col, ano):
  instituicoes = df[col].value_counts()
  fig = go.Figure(data=[go.Bar(x=instituicoes.index, y=instituicoes)])

  fig.update_layout(
      title=f'Instituições de ensino dos alunos - {ano}',
      xaxis_title='Instituição',
      yaxis_title='Quantidade'
  )

  fig.show()

In [7]:
plot_instituicao_ensino(df_2020, 'INSTITUICAO_ENSINO_ALUNO', 2020)

In [8]:
plot_instituicao_ensino(df_2021, 'INSTITUICAO_ENSINO_ALUNO', 2021)

In [10]:
def plot_histograma(df, col, ano):
  fig = px.histogram(df[col], title=f'Distribuição do indicador {col} para {ano}', nbins=50, marginal='rug')

  fig.update_layout(
      xaxis_title='Valor',
      yaxis_title='Frequência',
      height=600
  )

  fig.show()

In [12]:
plot_histograma(df_2020, 'INDE', 2020)

In [13]:
plot_histograma(df_2021, 'INDE', 2021)

In [14]:
plot_histograma(df_2022, 'INDE', 2022)

In [17]:
  def plot_histograma_comparativo(col):
    fig = px.histogram(data_frame=df, x=col, color='ANO', title=f'Distribuição do indicador {col} comparativo', nbins=50, marginal='rug')

    fig.update_layout(
        xaxis_title='Valor',
        yaxis_title='Frequência',
        height=600
    )

    fig.show()

In [18]:
plot_histograma_comparativo('INDE')

In [22]:
def plot_boxplot_indicador_por_ano(df, col, ano):
  fig = px.box(y=df[col], points="all", title=f'Distribuição do {col} para {ano}')

  fig.update_layout(
      yaxis_title='Valor'
  )

  fig.update_yaxes(dtick=1)

  fig.show()

In [23]:
def plot_boxplot_indicador_por_ano_comparativo(df, col):
  fig = px.box(data_frame=df, x='ANO', y=col, points="all", title=f'Distribuição do {col} comparativa', color='ANO')

  fig.update_layout(
      yaxis_title='Valor'
  )

  fig.update_yaxes(dtick=1)

  fig.show()

In [24]:
def plot_boxplot_analise_indicador(col):
  plot_boxplot_indicador_por_ano(df_2020, col, 2020)
  plot_boxplot_indicador_por_ano(df_2021, col, 2021)
  plot_boxplot_indicador_por_ano(df_2022, col, 2022)
  plot_boxplot_indicador_por_ano_comparativo(df, col)

In [25]:
plot_boxplot_analise_indicador('INDE')

In [26]:
plot_boxplot_analise_indicador('IAN')

In [27]:
plot_boxplot_analise_indicador('IAA')

In [28]:
plot_boxplot_analise_indicador('IEG')

In [29]:
plot_boxplot_analise_indicador('IPS')

In [30]:
plot_boxplot_analise_indicador('IDA')

In [31]:
plot_boxplot_analise_indicador('IPP')

In [32]:
plot_boxplot_analise_indicador('IPV')